In [365]:

!pip install opendatasets --upgrade --quiet
!pip install pandas 
!pip install imbalanced-learn
!pip install numpy 
!pip install pickle
!pip install seaborn
!pip install matplotlib.pyplot
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot
Defaulting to user installation because normal site-packages is not writeable


In [366]:

## REQUIRED LIBRARIES
# For data wrangling 
import numpy as np
import pandas as pd
import pickle 
import sklearn

# For visualisation
import matplotlib.pyplot as plt

In [367]:
  # Direct link to the CSV file on Google Drive
csv_url = 'https://drive.google.com/uc?id=1eMGf7o8nDF18gcgFnArI-2efU105FFg7'
# Read the CSV file from the URL
df = pd.read_csv(csv_url,parse_dates=['date'], index_col='date')
df.shape


(43800, 8)

In [368]:
df.head()

,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
date,,,,,,,,
2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,SE,1.79,0,0
2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,SE,2.68,0,0
2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,SE,3.57,0,0
2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,SE,5.36,1,0
2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,SE,6.25,2,0


In [369]:
# Save the DataFrame to a Pickle file in the data folder so easily accessed in all folders

with open('../data/raw_data.pickle', 'wb') as f:
    pickle.dump(df, f)

In [370]:
with open('../data/raw_data.pickle', 'rb') as f:
    df = pickle.load(f)

In [371]:
df.shape


(43800, 8)

In [372]:
df.head()

,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
date,,,,,,,,
2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,SE,1.79,0,0
2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,SE,2.68,0,0
2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,SE,3.57,0,0
2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,SE,5.36,1,0
2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,SE,6.25,2,0


In [373]:
list(df.columns)

['pollution', 'dew', 'temp', 'press', 'wnd_dir', 'wnd_spd', 'snow', 'rain']

In [374]:
df.describe()

,pollution,dew,temp,press,wnd_spd,snow,rain
count,43800.000000,43800.000000,43800.000000,43800.000000,43800.000000,43800.000000,43800.000000
mean,94.013516,1.828516,12.459041,1016.447306,23.894307,0.052763,0.195023
std,92.252276,14.429326,12.193384,10.271411,50.022729,0.760582,1.416247
min,0.000000,-40.000000,-19.000000,991.000000,0.450000,0.000000,0.000000
25%,24.000000,-10.000000,2.000000,1008.000000,1.790000,0.000000,0.000000
50%,68.000000,2.000000,14.000000,1016.000000,5.370000,0.000000,0.000000
75%,132.250000,15.000000,23.000000,1025.000000,21.910000,0.000000,0.000000
max,994.000000,28.000000,42.000000,1046.000000,585.600000,27.000000,36.000000


In [375]:
df.isnull().sum()

pollution    0
dew          0
temp         0
press        0
wnd_dir      0
wnd_spd      0
snow         0
rain         0
dtype: int64

In [376]:

# Get unique count for each variable
df.nunique()

pollution     581
dew            69
temp           64
press          60
wnd_dir         4
wnd_spd      2788
snow           28
rain           37
dtype: int64

In [377]:
df['wnd_dir'].unique()

array(['SE', 'cv', 'NW', 'NE'], dtype=object)

In [378]:
# Define the mapping dictionary
mapping = {'NE': 0, 'SE': 1, 'NW': 2, 'cv': 3}
# Replace the string values with numerical values
df['wnd_dir'] = df['wnd_dir'].map(mapping)

In [379]:
df.head()

,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
date,,,,,,,,
2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,1,1.79,0,0
2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,1,2.68,0,0
2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,1,3.57,0,0
2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,1,5.36,1,0
2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,1,6.25,2,0


In [380]:
#Splitting the dataset so that we treat 6 months of data as unseen to verify generalisability. 
n_records = len(df)  # Total number of samples
split_index = int(n_records * 0.9)  # Split at 90%

df_train, df_test = df[:split_index], df[split_index:]

df_train.shape,df_test.shape

((39420, 8), (4380, 8))

MinMax scaling transforms features to scale them within a specific range, (-1, 1). Where by using negative values we can give weight We scale with respect to the training minMax values to avoid exposing test data to the training set. 

In [381]:
from sklearn.preprocessing import MinMaxScaler
from pickle import dump

scaler_mv = MinMaxScaler()
df_train_mv = df_train.copy()
df_test_mv = df_test.copy()

# Define the columns to scale for multivariate
columns = (['pollution', 'dew', 'temp', 'press', 'wnd_spd', 
            'snow', 'rain' , "wnd_dir"])

#Scale the selected columns to the range 0-1. 
# We allow the same scaler to be used for test set as this has no risk of data leakage

df_train_mv[columns] = scaler_mv.fit_transform(df_train_mv[columns])
df_test_mv[columns] = scaler_mv.transform(df_test_mv[columns])


#Save the scaler
dump(scaler_mv, open('../data/scaler_mv.pickle', 'wb'))



We need a scaler that only acts on pollution column for univariate 

In [383]:
scaler_uv =MinMaxScaler()
df_train_uv = df_train[['pollution']].copy()
df_test_uv = df_test[['pollution']].copy()


df_train_uv['pollution'] = scaler_uv.fit_transform(df_train_uv)
df_test_uv['pollution'] = scaler_uv.transform(df_test_uv)


#Save the scaler
dump(scaler_uv, open('../data/scaler_uv.pickle', 'wb'))



In [ ]:
with open('../data/train_data.pickle', 'wb') as f:
    pickle.dump(df_train_mv, f)

with open('../data/test_data.pickle', 'wb') as f:
    pickle.dump(df_test_mv, f)